In [2]:

import PySAM.Pvwattsv8 as pv
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

# Define paths relative to the notebook
project_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
weather_file_path = os.path.join(project_dir, "data", "Bonfire_2025.epw")  # Using underscore instead of hyphen

# Check if weather file exists
if not os.path.exists(weather_file_path):
    print(f"Warning: Weather file not found at {weather_file_path}")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Project directory: {project_dir}")
    
    # Try alternative filenames
    alternatives = [
        os.path.join(project_dir, "data", "Bonfire-2025.epw"),  # with hyphen
        os.path.join(project_dir, "data", "bonfire_2025.epw"),  # lowercase
        os.path.join(project_dir, "data", "bonfire-2025.epw"),  # lowercase with hyphen
        os.path.join(project_dir, "data", "Bonfire.epw")        # just name
    ]
    
    for alt_path in alternatives:
        if os.path.exists(alt_path):
            weather_file_path = alt_path
            print(f"Found alternative weather file at: {weather_file_path}")
            break
    
    # List files in the data directory to help troubleshoot
    data_dir = os.path.join(project_dir, "data")
    if os.path.exists(data_dir):
        print("\nFiles in data directory:")
        for file in os.listdir(data_dir):
            if file.endswith('.epw'):
                print(f"- {file}")
else:
    print(f"Weather file found: {weather_file_path}")

# Create a PV system model
pv_system = pv.default("PVWattsNone")

# Set the weather file
pv_system.SolarResource.solar_resource_file = weather_file_path

# Set system parameters for Taggerty site
# System specifications
pv_system.SystemDesign.system_capacity = 10  # 10 kW DC as specified
pv_system.SystemDesign.dc_ac_ratio = 1.2
pv_system.SystemDesign.inv_eff = 96  # %
pv_system.SystemDesign.losses = 14.08  # Default system losses %

# Set location-specific parameters for Taggerty
pv_system.SystemDesign.tilt = 10  # 10 degrees tilt as specified
pv_system.SystemDesign.azimuth = 18  # 18 degrees from north towards east (for Australia)

# Set module type (0=standard, 1=premium, 2=thin film)
pv_system.SystemDesign.module_type = 0  # Standard modules

# Set array type (0=fixed, 1=1-axis, 2=2-axis)
pv_system.SystemDesign.array_type = 0  # Fixed array

# Implement basic shading with monthly shading factors (more shading in winter)
# For Southern Hemisphere: more shading in June-August (winter)
try:
    # Initial guess for monthly shading factors (Jan to Dec)
    # 1.0 means no shading, lower values = more shading
    monthly_shading = [0.95, 0.95, 0.93, 0.90, 0.85, 0.80, 0.80, 0.85, 0.90, 0.93, 0.95, 0.95]
    pv_system.SolarResource.shading_factor_beam = monthly_shading
    print("Applied monthly shading factors (initial guess):", monthly_shading)
    print("These will be optimized in the comparison script.")
except AttributeError:
    print("Monthly shading factor setting not available in this version of PySAM")
    # If monthly shading isn't available, we can increase the system losses instead
    pv_system.SystemDesign.losses += 5  # Add 5% to losses to account for shading
    print("Added 5% to system losses to account for shading")

# Execute the simulation
print("Running PVWatts simulation...")
pv_system.execute()

# Get the results
ac_output = pv_system.Outputs.ac
dc_output = pv_system.Outputs.dc

# Check if we have 8760 hours of data (full year)
print(f"Number of hourly values: {len(ac_output)}")

# Create a DataFrame with hourly results for 2024
start_date = datetime(2024, 1, 1, 0, 0)  # Jan 1, 2024 at 00:00
hours = [start_date + timedelta(hours=i) for i in range(len(ac_output))]

hourly_results = pd.DataFrame({
    'datetime': hours,
    'ac_output_kw': ac_output,  # AC power output (kW)
    'dc_output_kw': dc_output,  # DC power output (kW)
})

# Set datetime as index for resampling
hourly_results.set_index('datetime', inplace=True)

# Apply grid export limit of 3.5 kW
hourly_results['ac_output_limited_kw'] = hourly_results['ac_output_kw'].clip(upper=3.5)

# Resample to half-hourly intervals using linear interpolation
print("Resampling to half-hourly intervals using linear interpolation...")
half_hourly_results = hourly_results.resample("30min").interpolate(method='linear')

# Reset index to make datetime a column again
half_hourly_results.reset_index(inplace=True)

# Add month, day, hour, and minute columns for easier filtering
half_hourly_results['month'] = half_hourly_results['datetime'].dt.month
half_hourly_results['day'] = half_hourly_results['datetime'].dt.day
half_hourly_results['hour'] = half_hourly_results['datetime'].dt.hour
half_hourly_results['minute'] = half_hourly_results['datetime'].dt.minute

# Display the first few rows
print("\nFirst few rows of the half-hourly results:")
print(half_hourly_results.head())

# Save the results to CSV
output_dir = os.path.join(project_dir, "outputs")
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "taggerty_pv_baseline_half_hourly.csv")
half_hourly_results.to_csv(output_file, index=False)
print(f"\nHalf-hourly solar PV outputs saved to: {output_file}")

# Print a simple annual summary
annual_energy_kwh = half_hourly_results['ac_output_limited_kw'].sum() / 2  # Divide by 2 since we have half-hour values
annual_energy_mwh = annual_energy_kwh / 1000
capacity_factor = annual_energy_kwh / (10 * 8760) * 100  # 10 kW system, 8760 hours in a year

print(f"\nTotal Annual Generation: {annual_energy_kwh:.2f} kWh ({annual_energy_mwh:.2f} MWh)")
print(f"Capacity Factor: {capacity_factor:.2f}%")
print(f"Average Daily Generation: {annual_energy_kwh/365:.2f} kWh/day")

# Calculate monthly generation for analysis
monthly_generation = half_hourly_results.groupby('month')['ac_output_limited_kw'].sum() / 2  # Divide by 2 for half-hourly values
monthly_generation_kwh = monthly_generation.to_dict()

print("\nMonthly Generation (kWh):")
for month, generation in monthly_generation_kwh.items():
    month_name = datetime(2024, month, 1).strftime('%B')
    print(f"{month_name}: {generation:.2f} kWh")

# Save parameter information for the optimization code to use
params = {
    'system_capacity': 10,
    'dc_ac_ratio': 1.2,
    'inv_eff': 96,
    'losses': 14.08,
    'tilt': 10,
    'azimuth': 18,
    'module_type': 0,
    'array_type': 0,
    'monthly_shading': monthly_shading
}

import json
params_file = os.path.join(output_dir, "baseline_parameters.json")
with open(params_file, 'w') as f:
    json.dump(params, f, indent=4)
print(f"\nSaved baseline parameters to: {params_file}")

print("\nBaseline simulation complete. Run the optimization script next to compare with actual data.")

Weather file found: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/Bonfire_2025.epw
Monthly shading factor setting not available in this version of PySAM
Added 5% to system losses to account for shading
Running PVWatts simulation...
Number of hourly values: 8760
Resampling to half-hourly intervals using linear interpolation...

First few rows of the half-hourly results:
             datetime  ac_output_kw  dc_output_kw  ac_output_limited_kw  \
0 2024-01-01 00:00:00           0.0           0.0                   0.0   
1 2024-01-01 00:30:00           0.0           0.0                   0.0   
2 2024-01-01 01:00:00           0.0           0.0                   0.0   
3 2024-01-01 01:30:00           0.0           0.0                   0.0   
4 2024-01-01 02:00:00           0.0           0.0                   0.0   

   month  day  hour  minute  
0      1    1     0       0  
1      1    1     0      30  
2      1    1     1       0  
3      1    1     1      30  
4      1   